In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
%pwd

'f:\\Generative_AI\\Interview_question_creator_project\\research'

In [6]:
%cd ..

f:\Generative_AI\Interview_question_creator_project


In [7]:
file_path ="F:\Generative_AI\Interview_question_creator_project\data\SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [8]:
data

[Document(page_content='', metadata={'source': 'F:\\Generative_AI\\Interview_question_creator_project\\data\\SDG.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': 'F:\\Generative_AI\\Interview_question_creator_project\\data\\SDG.pdf', 'page': 1}),
 Document(page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called

In [9]:
len(data)

24

In [10]:
question_gen = ""

for page in data:
    question_gen += page.page_content

In [11]:
from langchain.text_splitter import TokenTextSplitter

In [12]:
splitter_ques_gen = TokenTextSplitter(
    model_name= "gpt2",
    chunk_size= 10000,
    chunk_overlap = 200
)

In [13]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [14]:
len(chunk_ques_gen)

1

In [15]:
from langchain.docstore.document import Document

In [16]:
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [17]:
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt2',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [18]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [19]:
len(document_answer_gen)

5

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\ROBERT\anaconda3\envs\interview\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
llm_ques_gen_pipeline = ChatGoogleGenerativeAI(
    model = 'gemini-pro',
    temperature = 0.3
)


In [22]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [23]:
from langchain.prompts import PromptTemplate

In [24]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [25]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [26]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [27]:
from langchain.chains.summarize import load_summarize_chain

In [28]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [29]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)

c:\Users\ROBERT\anaconda3\envs\interview\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew  
that earthquakes and floods were inevitable, but that the high death  
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders f

In [30]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [31]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [32]:
from langchain.vectorstores import FAISS

In [33]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

In [34]:
llm_answer_gen = ChatGoogleGenerativeAI(temperature=0.1, model="gemini-1.5-pro-latest")

In [35]:
ques

"**Question 1:**\n\nWhat is the primary goal of the Sustainable Development Goals (SDGs)?\n\n**Question 2:**\n\nHow many countries have agreed to the SDGs?\n\n**Question 3:**\n\nWhat is the target year for achieving the SDGs?\n\n**Question 4:**\n\nWhat is the current status of extreme poverty in the world?\n\n**Question 5:**\n\nWhat is the goal for hunger and malnutrition by 2030?\n\n**Question 6:**\n\nWhat is the leading cause of death for adolescents in sub-Saharan Africa?\n\n**Question 7:**\n\nWhat is the current enrolment rate in developing regions for primary education?\n\n**Question 8:**\n\nWhat is the main challenge in providing access to affordable, reliable energy for all?\n\n**Question 9:**\n\nWhat is the goal for reducing inequality within and among countries?\n\n**Question 10:**\n\nWhat is the estimated percentage of the world's population that will live in cities by 2050?\n\n**Question 11:**\n\nWhat is the target for reducing per capita food waste globally?\n\n**Question 1

In [36]:
ques_list = ques.split('\n')

In [37]:
ques_list

['**Question 1:**',
 '',
 'What is the primary goal of the Sustainable Development Goals (SDGs)?',
 '',
 '**Question 2:**',
 '',
 'How many countries have agreed to the SDGs?',
 '',
 '**Question 3:**',
 '',
 'What is the target year for achieving the SDGs?',
 '',
 '**Question 4:**',
 '',
 'What is the current status of extreme poverty in the world?',
 '',
 '**Question 5:**',
 '',
 'What is the goal for hunger and malnutrition by 2030?',
 '',
 '**Question 6:**',
 '',
 'What is the leading cause of death for adolescents in sub-Saharan Africa?',
 '',
 '**Question 7:**',
 '',
 'What is the current enrolment rate in developing regions for primary education?',
 '',
 '**Question 8:**',
 '',
 'What is the main challenge in providing access to affordable, reliable energy for all?',
 '',
 '**Question 9:**',
 '',
 'What is the goal for reducing inequality within and among countries?',
 '',
 '**Question 10:**',
 '',
 "What is the estimated percentage of the world's population that will live in cit

In [38]:
from langchain.chains import RetrievalQA

In [39]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())

In [40]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  **Question 1:**
Answer:  Please provide me with the first question. I am ready to answer it using the context you have provided. 

--------------------------------------------------\n\n
Question:  
Answer:  I'm sorry, but the provided text doesn't contain the answer to your question. Can you please provide me with the question you have? 

--------------------------------------------------\n\n
Question:  What is the primary goal of the Sustainable Development Goals (SDGs)?
Answer:  The primary goal of the Sustainable Development Goals (SDGs) is to rid the world of poverty and hunger, and to ensure safety from the worst effects of climate change by 2030. 

--------------------------------------------------\n\n
Question:  


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Answer:  I'm sorry, but the provided text does not contain the answer to your question. Can you please provide me with the question you have? 

--------------------------------------------------\n\n
Question:  **Question 2:**
Answer:  Please provide me with the second question. I am ready to answer it using the context you have provided. 

--------------------------------------------------\n\n
Question:  


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Answer:  I'm sorry, but the provided text does not contain the answer to your question. Can you please provide me with the question you have? 

--------------------------------------------------\n\n
Question:  How many countries have agreed to the SDGs?
Answer:  193 countries agreed on the Sustainable Development Goals. 

--------------------------------------------------\n\n
Question:  
Answer:  I'm sorry, but the provided text does not contain the answer to your question. Can you please provide me with the question you would like answered? 

--------------------------------------------------\n\n
Question:  **Question 3:**
Answer:  I'm sorry, but I need a full question to be able to provide an answer. Please provide the complete question. 

--------------------------------------------------\n\n
Question:  


...

Answer:  I'm sorry, but the provided text doesn't contain the answer to your question. Can you please provide me with the question you have? 

--------------------------------------------------\n\n
Question:  What is the main threat to forests and other terrestrial ecosystems?
Answer:  The text states that arable land is disappearing 30 to 35 times faster than it has historically, deserts are spreading, and animal breeds are going extinct. It does not, however, explicitly state the main threat to forests and other terrestrial ecosystems. 

--------------------------------------------------\n\n
Question:  
Answer:  I'm sorry, but the provided text doesn't contain the answer to your question. Can you please provide the question you have? 

--------------------------------------------------\n\n
Question:  **Question 14:**
Answer:  I'm sorry, but the provided text does not contain a 14th question for me to answer. 

--------------------------------------------------\n\n
Question:  


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Answer:  I'm sorry, but the provided text does not contain the answer to your question. Can you please provide me with your question? 

--------------------------------------------------\n\n
Question:  What is the goal for promoting peace and justice for sustainable development?
Answer:  The goal of peace and justice for sustainable development aims to reduce all forms of violence and proposes that governments and communities find lasting solutions to conflict and insecurity. This includes strengthening the rule of law, reducing the flow of illicit arms, and bringing developing countries more into the center of institutions of global governance. 

--------------------------------------------------\n\n
Question:  
Answer:  I'm sorry, but the provided text doesn't contain the answer to your question. It describes the Sustainable Development Goals but doesn't seem to include a specific question from a user. 

Please provide me with the user's question, and I'll do my best to answer it usi

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Answer:  The final goal of the SDGs is to **strengthen the means of implementation and revitalize the global partnership for sustainable development.**

This goal aims to achieve the other goals by:

* **Fostering international cooperation:**  It emphasizes the need for countries to work together, share resources, and support each other in achieving the SDGs.
* **Promoting trade and technology transfer:**  It encourages developed countries to help developing countries access the technology and resources they need to achieve sustainable development.
* **Mobilizing financial resources:** It calls for increased financial support from developed countries to developing countries to implement the SDGs.
* **Improving data collection and monitoring:** It stresses the importance of tracking progress towards the SDGs and using data to inform policy decisions.

Essentially, the final goal recognizes that achieving the SDGs requires a global effort and aims to create a framework for collaboration 